In [2]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import re
import os
import glob
import shutil
from openpyxl import Workbook
from pathlib import Path
from datetime import datetime, timedelta
os.getcwd()

'/Users/satoshiido/Documents/programming/RA/pharma'

In [3]:
# create the input_dir（input directory）
source_path = os.path.dirname(os.path.abspath('__file__'))
# source_path = '/scratch/bell/sido/m&a'
INPUT_DIR = os.path.join(source_path, 'data')

# if INPUT_DIR has not been created yet, create it
if not os.path.isdir(INPUT_DIR):
    os.mkdir(INPUT_DIR)

# output_dir(output directory) creation
OUTPUT_DIR = os.path.join(source_path, 'outputs')

# if OUTPUT_DIR has not been created yet, create it
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [4]:
# Once you run this code, comment it out
# move csv files to `data` directory(=folder)

# unique_dir_names = []
# for f in Path(f'{source_path}').rglob('*.csv'):
#     unique_dir_names.append(f)
# for g in Path(f'{source_path}').rglob('*.xlsx'):
#     unique_dir_names.append(g)

# for file in list(set(unique_dir_names)):
#     print(f'moved file: {file}')
#     shutil.move(f'{file}', f'{INPUT_DIR}')

moved file: /Users/satoshiido/Documents/programming/RA/pharma/temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.xlsx
moved file: /Users/satoshiido/Documents/programming/RA/pharma/~$temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.xlsx
moved file: /Users/satoshiido/Documents/programming/RA/pharma/name_matching/USPTOtoCRSP_Pharma_Dictionary.csv
moved file: /Users/satoshiido/Documents/programming/RA/pharma/USPTO_firm_list.xlsx
moved file: /Users/satoshiido/Documents/programming/RA/pharma/temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.csv
moved file: /Users/satoshiido/Documents/programming/RA/pharma/name_matching/same_orgname_dif_assigneeid_pharma.csv
moved file: /Users/satoshiido/Documents/programming/RA/pharma/name_matching/duplicated_cleaned_pharma.csv
moved file: /Users/satoshiido/Documents/programming/RA/pharma/name_matching/pharma-not-in-ma-list-20230416.csv


In [5]:
# Pandas function to let us read csv files without having to specify the directory
def read_csv(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    return pd.read_csv(path, **kwrgs)

# Polars function to let us read csv files wi`thout having to specify the directory
def read_csvpl(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    return pl.read_csv(path, **kwrgs)

# DATA IMPORT

In [15]:
#convert xlsx to csv
# firm_dir = pd.DataFrame(pd.read_excel(os.path.join(INPUT_DIR, "USPTO_firm_list.xlsx")))
# firm_dir.to_csv(os.path.join(INPUT_DIR, "uspto_firm_list.csv"), encoding='utf-8', index=False)
# temproster = pd.DataFrame(pd.read_excel(os.path.join(INPUT_DIR, "temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.xlsx")))
# temproster.to_csv(os.path.join(INPUT_DIR, "temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.csv"), encoding='utf-8', index=False)

In [55]:
firm_dir = read_csv('uspto_firm_list')
temproster = read_csv('temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist')

Load: /Users/satoshiido/Documents/programming/RA/pharma/data/uspto_firm_list.csv
Load: /Users/satoshiido/Documents/programming/RA/pharma/data/temproster2_allversion_targetTOmissingand_recapdatanotmissingOct2023_Firmlist.csv


In [56]:
firm_dir

,assignee_id,organization
0,000049db-1cf0-4735-8a71-5a44b04859a6,"Atlantech International, Inc."
1,000074b2-2134-459b-a481-26c72c2313ae,Shanghai Juge Electronics Technologies Co. Ltd.
2,0000c853-052b-4c30-acea-0c64e371349c,"Ringertown Innovations, LLC"
3,00011057-f995-4fea-b456-0a0c05f2d760,"Rackspace, LTD"
4,00014eb5-814b-41d2-b61c-142fc461d058,Dansk System Elektronik A/S
...,...,...
375441,ffffb054-33c5-4d9f-86c7-b92f4d771104,"Yamaichi New Technology Co., Ltd."
375442,ffffdac9-09bb-460d-b3b2-06842317a2d1,Citius Burotechnik GmbH
375443,ffffed49-446e-4c9e-b195-f3aad9f49757,"Glucan BioRenewables, LLC"
375444,fffffaaf-3d20-43c7-8e6d-7523a76d6305,AISing LTD.


In [57]:
temproster

,TargetName
0,3D Systems Corp
1,AB SCIEX
2,ACT Gene SA
3,ADAC Laboratories
4,AECI Ltd
...,...
1473,iviGene Corp
1474,microcuff GmbH
1475,nanoTechnology Systems Pty Ltd
1476,picoSpin LLC


# PREPROCESS

In [58]:
# Check for non-string or missing values in the 'organization' column
firm_dir[firm_dir['organization'].apply(lambda x: not isinstance(x, str))]

,assignee_id,organization
106094,4852ee47-e206-4b18-8fbd-3027c7020c3c,NaN
279157,be8d184f-ad1e-4cce-ba25-9588bf36c054,NaN
326732,ded97c9f-f562-49f6-bbfa-7591b6c5d6d1,NaN


In [59]:
# fill NaN values with an empty string
firm_dir['organization'].fillna("", inplace=True)

In [60]:
# regular expression to remove the following words from the company name
def clean_company_name(name):
    # extend the regular expression to include 'A/S' and use a non-capturing group for postfixes
    # the regex will handle cases like "Co. Ltd." to remove following words    
    postfix_pattern = re.compile(
        r'(,?\s*(Inc\.?|Ltd\.?|LLC|LTD|GmbH|AG|Co\.|Corporation|Group|S\.A\.|S\.P\.A\.|S\.A\.S\.|S\.R\.L\.|LLP|LP|'
        r'S\.L\.|Oy|Oyj|Zrt|Pty|Kg|Kgaa|N\.V\.|HF|BVBA|B\.V\.|S\.E\.|PLC|PBC|S\.C\.A\.|S\.E\.M\.C\.O\.|S\.E\.C\.|'
        r'A/S|Co\.\s+\S+|Limited\s+\S+)(\s*,.*|$))', re.IGNORECASE)

    # Substitute found patterns with an empty string
    cleaned_name = re.sub(postfix_pattern, "", name).strip()
    cleaned_name = cleaned_name.lower()
    
    return cleaned_name

In [61]:
# apply the updated function to the dataframe
firm_dir["cleaned_organization"] = firm_dir["organization"].apply(clean_company_name)
temproster["cleaned_TargetName"] = temproster["TargetName"].apply(clean_company_name)

# display the results
firm_dir[["assignee_id", "organization", "cleaned_organization"]].head(20)

,assignee_id,organization,cleaned_organization
0,000049db-1cf0-4735-8a71-5a44b04859a6,"Atlantech International, Inc.",atlantech international
1,000074b2-2134-459b-a481-26c72c2313ae,Shanghai Juge Electronics Technologies Co. Ltd.,shanghai juge electronics technologies
2,0000c853-052b-4c30-acea-0c64e371349c,"Ringertown Innovations, LLC",ringertown innovations
3,00011057-f995-4fea-b456-0a0c05f2d760,"Rackspace, LTD",rackspace
4,00014eb5-814b-41d2-b61c-142fc461d058,Dansk System Elektronik A/S,dansk system elektronik
5,000153d6-df39-497a-9bd5-c6b1cd2f4f7c,"Agile Materials and Technologies, Inc.",agile materials and technologies
6,00018413-46d8-4a6a-90c4-153603f06b28,LIGHTECH ELECTRONIC INDUSTRIES LTD.,lightech electronic industries
7,0001ac8b-4cac-495f-bce4-1823193fe0cc,"PWC Industries, Inc.",pwc industries
8,0001b532-a5a1-4d12-af98-281d54efd682,"SAMSUNG DSPLAY CO., LTD.",samsung dsplay
9,0001ed26-098e-4672-9407-68f19e84afd0,"Speedling, Inc.",speedling


In [62]:
temproster.head(20)

,TargetName,cleaned_TargetName
0,3D Systems Corp,3d systems corp
1,AB SCIEX,ab sciex
2,ACT Gene SA,act gene sa
3,ADAC Laboratories,adac laboratories
4,AECI Ltd,aeci
5,AEG Elektrofotografie GmbH,aeg elektrofotografie
6,AFx Inc,afx
7,ALIGN Pharmaceuticals LLC,align pharmaceuticals
8,ALZA Corp,alza corp
9,AMAG Pharmaceuticals Inc,amag pharmaceuticals


In [63]:
temproster

,TargetName,cleaned_TargetName
0,3D Systems Corp,3d systems corp
1,AB SCIEX,ab sciex
2,ACT Gene SA,act gene sa
3,ADAC Laboratories,adac laboratories
4,AECI Ltd,aeci
...,...,...
1473,iviGene Corp,ivigene corp
1474,microcuff GmbH,microcuff
1475,nanoTechnology Systems Pty Ltd,nanotechnology systems pty
1476,picoSpin LLC,picospin


In [23]:
def data_cleaning(df):
    #remove words in brackets
    df = df.apply(lambda c: re.sub("\(.*?\)","()",str(c)))
    df = df.apply(lambda c: re.sub(r"[\([{})\]]", "", c))

    #remove suffix
    df = df.str.lower()
    df = df.apply(lambda c: c.replace(" corporation", ""))
    df = df.apply(lambda c: c.replace(" company", ""))
    df = df.apply(lambda c: c.replace(" kaisha", ""))
    df = df.apply(lambda c: c.replace(", the", ""))
    df = df.apply(lambda c: c.replace(" ug", ""))
    df = df.apply(lambda c: c.replace(" oao", ""))
    df = df.apply(lambda c: c.replace(" pt", ""))
    df = df.apply(lambda c: c.replace(" ny", ""))
    df = df.apply(lambda c: c.replace(" as", ""))
    df = df.apply(lambda c: c.replace(" se", ""))
    df = df.apply(lambda c: c.replace(" sl", ""))
    df = df.apply(lambda c: c.replace(" ab", ""))
    df = df.apply(lambda c: c.replace(" ag", ""))
    df = df.apply(lambda c: c.replace("-ag", ""))
    df = df.apply(lambda c: c.replace(" lc", ""))
    df = df.apply(lambda c: c.replace(" nv", ""))
    df = df.apply(lambda c: c.replace(" sasu", ""))
    df = df.apply(lambda c: c.replace(" sas", ""))
    df = df.apply(lambda c: c.replace(" sa", ""))
    df = df.apply(lambda c: c.replace(" aps", ""))
    df = df.apply(lambda c: c.replace(" asa", ""))
    df = df.apply(lambda c: c.replace(" spa", ""))
    df = df.apply(lambda c: c.replace(" sar", ""))
    df = df.apply(lambda c: c.replace(" sia", ""))
    
    df = df.apply(lambda c: c.replace(" co. ltd", ""))
    df = df.apply(lambda c: c.replace(" co ltd", ""))
    df = df.apply(lambda c: c.replace(" co.", ""))
    df = df.apply(lambda c: c.replace(" m sdn bhd", ""))
    df = df.apply(lambda c: c.replace(" sdn bhd", ""))
    df = df.apply(lambda c: c.replace("gmbh & co", ""))
    df = df.apply(lambda c: c.replace("gmnh", ""))
    df = df.apply(lambda c: c.replace(" kgaa", ""))
    df = df.apply(lambda c: c.replace(" kg", ""))
    df = df.apply(lambda c: c.replace("gmbh", ""))
    df = df.apply(lambda c: c.replace(" bvba", ""))
    df = df.apply(lambda c: c.replace(" b.v.", ""))
    df = df.apply(lambda c: c.replace(" b. v.", ""))
    df = df.apply(lambda c: c.replace(" bv", ""))
    df = df.apply(lambda c: c.replace(" limited", ""))
    df = df.apply(lambda c: c.replace(" corp", ""))
    df = df.apply(lambda c: c.replace(" & co.", ""))
    df = df.apply(lambda c: c.replace(" & c.s.n.c.", ""))
    df = df.apply(lambda c: c.replace(" & c. s.n.c.", ""))
    df = df.apply(lambda c: c.replace(" a/s", ""))
    df = df.apply(lambda c: c.replace(" a.s.", ""))
    df = df.apply(lambda c: c.replace(" de cv", ""))
    df = df.apply(lambda c: c.replace(" s.a.s.u.", ""))
    df = df.apply(lambda c: c.replace(" s.a.s.", ""))
    df = df.apply(lambda c: c.replace("(s.a.s.)", ""))
    df = df.apply(lambda c: c.replace(" s.a.b.", ""))
    df = df.apply(lambda c: c.replace(" s.a.p.i.", ""))
    df = df.apply(lambda c: c.replace(" s.a.", ""))
    df = df.apply(lambda c: c.replace(" s.a", ""))
    df = df.apply(lambda c: c.replace(" s.r.o.", ""))
    df = df.apply(lambda c: c.replace(" s.r.o", ""))
    df = df.apply(lambda c: c.replace(" s.p.a.", ""))
    df = df.apply(lambda c: c.replace(" p.a.", ""))
    df = df.apply(lambda c: c.replace(" s.a.r.l.", ""))
    df = df.apply(lambda c: c.replace(" sarl", ""))
    df = df.apply(lambda c: c.replace("s.r.l.", ""))
    df = df.apply(lambda c: c.replace("s.r.l", ""))
    df = df.apply(lambda c: c.replace(" srl", ""))
    df = df.apply(lambda c: c.replace(" ksaa", ""))
    df = df.apply(lambda c: c.replace(" k.s.a.a.", ""))
    df = df.apply(lambda c: c.replace(" d.o.o.", ""))
    df = df.apply(lambda c: c.replace(" s.l.p.", ""))
    df = df.apply(lambda c: c.replace(" s.l.u.", ""))
    df = df.apply(lambda c: c.replace(" l.p.", ""))
    df = df.apply(lambda c: c.replace(" l.p", ""))
    df = df.apply(lambda c: c.replace(" s.l.", ""))
    df = df.apply(lambda c: c.replace(" s.l", ""))
    df = df.apply(lambda c: c.replace(" s/a", ""))
    df = df.apply(lambda c: c.replace(" gbr", ""))
    df = df.apply(lambda c: c.replace(" oyj", ""))
    df = df.apply(lambda c: c.replace(" oy", ""))
    df = df.apply(lambda c: c.replace(" l/s", ""))
    df = df.apply(lambda c: c.replace(" n.v.", ""))
    df = df.apply(lambda c: c.replace(" nv", ""))
    df = df.apply(lambda c: c.replace(" hf", ""))
    df = df.apply(lambda c: c.replace(" e.v.", ""))
    df = df.apply(lambda c: c.replace(" de c.v.", ""))
    df = df.apply(lambda c: c.replace(" de .c.v.", ""))
    df = df.apply(lambda c: c.replace(" ulc", ""))
    df = df.apply(lambda c: c.replace(" s.c. a r.l.", ""))
    df = df.apply(lambda c: c.replace(" pbc", ""))
    df = df.apply(lambda c: c.replace(" llc", ""))
    df = df.apply(lambda c: c.replace(" l. l. c.", ""))
    df = df.apply(lambda c: c.replace(" l.c.", ""))
    df = df.apply(lambda c: c.replace(" ltd", ""))
    df = df.apply(lambda c: c.replace(" l.t.d.", ""))
    df = df.apply(lambda c: c.replace(" l.t.d", ""))
    df = df.apply(lambda c: c.replace(" lt.d", ""))
    df = df.apply(lambda c: c.replace(" lllp", ""))
    df = df.apply(lambda c: c.replace(" llp", ""))
    df = df.apply(lambda c: c.replace(" lda", ""))
    df = df.apply(lambda c: c.replace(" lp", ""))
    df = df.apply(lambda c: c.replace(".ltd", ""))
    df = df.apply(lambda c: c.replace(" inc.", ""))
    df = df.apply(lambda c: c.replace(" inc", ""))
    df = df.apply(lambda c: c.replace(" l.l.c.", ""))
    df = df.apply(lambda c: c.replace(" l l c", ""))
    df = df.apply(lambda c: c.replace(" pte", ""))
    df = df.apply(lambda c: c.replace(" s.e.m.c.o.", ""))
    df = df.apply(lambda c: c.replace(" s.e.c.", ""))
    df = df.apply(lambda c: c.replace(" pllc", ""))
    df = df.apply(lambda c: c.replace(" plc", ""))
    df = df.apply(lambda c: c.replace(" p.l.c.", ""))
    df = df.apply(lambda c: c.replace(" pvt", ""))
    df = df.apply(lambda c: c.replace(" pcl", ""))
    df = df.apply(lambda c: c.replace(" sp. z o. o.", ""))
    df = df.apply(lambda c: c.replace(" sp z o o", ""))
    df = df.apply(lambda c: c.replace(" sp. z o. o", ""))
    df = df.apply(lambda c: c.replace(" sp. z o.o", ""))
    df = df.apply(lambda c: c.replace(" sp. zo.o", ""))
    df = df.apply(lambda c: c.replace(" sp.zo.o", ""))
    df = df.apply(lambda c: c.replace(" pty", ""))
    df = df.apply(lambda c: c.replace(" zrt", ""))
    df = df.apply(lambda c: c.replace(" co ", ""))
    df = df.apply(lambda c: c.replace(" l p", ""))

    #remove special characters
    df = df.apply(lambda c: c.replace(".", ""))
    df = df.apply(lambda c: c.replace(",", ""))
    df = df.apply(lambda c: c.replace("-", ""))
    df = df.apply(lambda c: c.replace("!", ""))
    df = df.apply(lambda c: c.replace(":", ""))
    df = df.apply(lambda c: c.replace("'", ""))
    df = df.apply(lambda c: c.replace("/", ""))
    df = df.apply(lambda c: c.replace("\\", ""))
    df = df.apply(lambda c: c.replace("*", ""))

    #remove special words
    df = df.apply(lambda c: c.rstrip() if c.split(" ")[-1] in ("the","ab", "co", "sa", "as", "ao", "oao", "ooo", "inc", "dac" "sro", "pc", "sb", "&", "amt") else c)
    df = df.apply(lambda c: c.replace("group", "grp"))
    df = df.apply(lambda c: c.replace("international", "intl"))
    df = df.apply(lambda c: c.replace(" and", " &"))

    #remove spaces
    df = df.apply(lambda c: c.strip())
    df = df.apply(lambda c: c.replace(" ", ""))
    return df


In [24]:
firm_dir['organization'] = data_cleaning(firm_dir['organization'])

In [27]:
pd.set_option('display.max_rows', 100)
firm_dir.head(100)

,assignee_id,organization
0,000049db-1cf0-4735-8a71-5a44b04859a6,atlantechintl
1,000074b2-2134-459b-a481-26c72c2313ae,shanghaijugeelectronicstechnologies
2,0000c853-052b-4c30-acea-0c64e371349c,ringertowninnovations
3,00011057-f995-4fea-b456-0a0c05f2d760,rackspace
4,00014eb5-814b-41d2-b61c-142fc461d058,dansksystemelektronik
5,000153d6-df39-497a-9bd5-c6b1cd2f4f7c,agilematerials&technologies
6,00018413-46d8-4a6a-90c4-153603f06b28,lightechelectronicindustries
7,0001ac8b-4cac-495f-bce4-1823193fe0cc,pwcindustries
8,0001b532-a5a1-4d12-af98-281d54efd682,samsungdsplay
9,0001ed26-098e-4672-9407-68f19e84afd0,speedling
